# ЛР3

# Проверка статистических гипотез о виде закона распределения

In [3]:
from random import uniform
from tabulate import tabulate
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np

In [1]:
def Y(x):
    return 2 / (2 + x)

A = -1
B = 5

## 1

In [7]:
n = 200
X = sorted(round(uniform(0, 1) * (B - A) + A, 4) for _ in range(n))
x_list = sorted([round(Y(X[i]), 3) for i in range(n)])
print(x_list)

[0.287, 0.29, 0.29, 0.292, 0.294, 0.298, 0.298, 0.298, 0.3, 0.303, 0.304, 0.308, 0.311, 0.313, 0.314, 0.314, 0.314, 0.318, 0.318, 0.318, 0.319, 0.321, 0.326, 0.326, 0.327, 0.327, 0.33, 0.335, 0.335, 0.338, 0.339, 0.339, 0.344, 0.345, 0.346, 0.346, 0.349, 0.352, 0.352, 0.353, 0.358, 0.359, 0.361, 0.362, 0.362, 0.363, 0.365, 0.369, 0.372, 0.374, 0.38, 0.384, 0.385, 0.388, 0.388, 0.39, 0.392, 0.394, 0.398, 0.4, 0.405, 0.411, 0.413, 0.415, 0.416, 0.417, 0.418, 0.424, 0.424, 0.43, 0.431, 0.431, 0.438, 0.439, 0.439, 0.441, 0.441, 0.441, 0.441, 0.444, 0.447, 0.449, 0.451, 0.452, 0.464, 0.467, 0.471, 0.479, 0.481, 0.486, 0.486, 0.488, 0.488, 0.49, 0.491, 0.492, 0.5, 0.502, 0.504, 0.504, 0.508, 0.508, 0.511, 0.514, 0.516, 0.518, 0.52, 0.52, 0.525, 0.53, 0.535, 0.537, 0.542, 0.543, 0.545, 0.55, 0.553, 0.555, 0.558, 0.561, 0.569, 0.571, 0.573, 0.573, 0.586, 0.59, 0.594, 0.596, 0.599, 0.6, 0.605, 0.627, 0.638, 0.657, 0.661, 0.661, 0.664, 0.676, 0.677, 0.681, 0.685, 0.693, 0.694, 0.703, 0.72, 0.724

## 2

In [5]:
n = 30
X = sorted(round(uniform(0, 1) * (B - A) + A, 4) for _ in range(n))
x_list = sorted([round(Y(X[i]), 3) for i in range(n)])
print(x_list)

[0.292, 0.292, 0.304, 0.307, 0.308, 0.318, 0.338, 0.356, 0.358, 0.372, 0.376, 0.387, 0.391, 0.535, 0.54, 0.565, 0.59, 0.6, 0.633, 0.753, 0.794, 0.796, 0.831, 0.855, 0.976, 1.117, 1.124, 1.229, 1.505, 1.523]


## 3

In [6]:
n = 50
X = sorted(round(uniform(0, 1) * (B - A) + A, 4) for _ in range(n))
x_list = sorted([round(Y(X[i]), 3) for i in range(n)])
print(x_list)

[0.288, 0.31, 0.313, 0.318, 0.324, 0.325, 0.326, 0.348, 0.35, 0.351, 0.36, 0.361, 0.362, 0.363, 0.368, 0.374, 0.378, 0.407, 0.407, 0.408, 0.414, 0.427, 0.43, 0.434, 0.44, 0.469, 0.492, 0.495, 0.5, 0.512, 0.522, 0.526, 0.602, 0.622, 0.643, 0.656, 0.71, 0.726, 0.761, 0.817, 0.93, 1.023, 1.05, 1.079, 1.344, 1.444, 1.582, 1.637, 1.776, 1.87]
